In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 12
batch_size = 512
device = 'cuda'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.318262577056885 | KNN Loss: 5.820803642272949 | CLS Loss: 1.497458815574646
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 5.080780982971191 | KNN Loss: 4.2764387130737305 | CLS Loss: 0.8043422698974609
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.642612934112549 | KNN Loss: 3.95671010017395 | CLS Loss: 0.6859029531478882
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 4.5135345458984375 | KNN Loss: 3.8869571685791016 | CLS Loss: 0.6265771985054016
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 4.441784858703613 | KNN Loss: 3.8211803436279297 | CLS Loss: 0.6206047534942627
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 4.345569133758545 | KNN Loss: 3.8533806800842285 | CLS Loss: 0.49218854308128357
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 4.419981002807617 | KNN Loss: 3.8789937496185303 | CLS Loss: 0.5409873127937317
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.247969150543213 | KNN Loss: 3.846813678741455 | C

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.79848575592041 | KNN Loss: 3.689203977584839 | CLS Loss: 0.10928184539079666
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.8146355152130127 | KNN Loss: 3.6692540645599365 | CLS Loss: 0.14538134634494781
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.789215564727783 | KNN Loss: 3.7060489654541016 | CLS Loss: 0.08316660672426224
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.8082547187805176 | KNN Loss: 3.6728105545043945 | CLS Loss: 0.13544407486915588
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.847170114517212 | KNN Loss: 3.7044966220855713 | CLS Loss: 0.1426735818386078
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.7556545734405518 | KNN Loss: 3.6470677852630615 | CLS Loss: 0.10858669877052307
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.7774887084960938 | KNN Loss: 3.6704304218292236 | CLS Loss: 0.10705823451280594
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.7957208156585693 | KNN Loss: 3.7

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.7533836364746094 | KNN Loss: 3.643889904022217 | CLS Loss: 0.1094936728477478
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.759463310241699 | KNN Loss: 3.6787896156311035 | CLS Loss: 0.08067367225885391
Epoch: 007, Loss: 3.7559, Train: 0.9793, Valid: 0.9760, Best: 0.9760
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.738450288772583 | KNN Loss: 3.6555938720703125 | CLS Loss: 0.08285630494356155
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.70348858833313 | KNN Loss: 3.6520161628723145 | CLS Loss: 0.05147242546081543
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.7704567909240723 | KNN Loss: 3.6611456871032715 | CLS Loss: 0.10931115597486496
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.7452759742736816 | KNN Loss: 3.672139883041382 | CLS Loss: 0.07313603907823563
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.708233594894409 | KNN Loss: 3.630563497543335 | CLS Loss: 0.07767011225223541
Epoch 8 / 200 | iterat

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.7627627849578857 | KNN Loss: 3.675680160522461 | CLS Loss: 0.08708266913890839
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.7342638969421387 | KNN Loss: 3.6771769523620605 | CLS Loss: 0.05708690732717514
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.7306833267211914 | KNN Loss: 3.6280019283294678 | CLS Loss: 0.1026814728975296
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.719848871231079 | KNN Loss: 3.638535737991333 | CLS Loss: 0.0813131183385849
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.70465350151062 | KNN Loss: 3.6479318141937256 | CLS Loss: 0.05672163888812065
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.710153579711914 | KNN Loss: 3.6282947063446045 | CLS Loss: 0.08185885101556778
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.698760509490967 | KNN Loss: 3.644787073135376 | CLS Loss: 0.05397352576255798
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.728572368621826 | KNN Loss: 3.6

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.6565840244293213 | KNN Loss: 3.6200175285339355 | CLS Loss: 0.03656653314828873
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.672551155090332 | KNN Loss: 3.6199028491973877 | CLS Loss: 0.052648186683654785
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.731665849685669 | KNN Loss: 3.6519761085510254 | CLS Loss: 0.07968978583812714
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.680635452270508 | KNN Loss: 3.608729600906372 | CLS Loss: 0.07190591841936111
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.754255771636963 | KNN Loss: 3.6525609493255615 | CLS Loss: 0.10169482231140137
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.693587064743042 | KNN Loss: 3.6085550785064697 | CLS Loss: 0.0850318968296051
Epoch: 014, Loss: 3.6998, Train: 0.9820, Valid: 0.9783, Best: 0.9814
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.724168300628662 | KNN Loss: 3.647566556930542 | CLS Loss: 0.07660174369812012
Epoch 15 /

...

Epoch 185 / 200 | iteration 140 / 171 | Total Loss: 3.5805492401123047 | KNN Loss: 3.5769805908203125 | CLS Loss: 0.003568677231669426
Epoch 185 / 200 | iteration 150 / 171 | Total Loss: 3.5757715702056885 | KNN Loss: 3.57362961769104 | CLS Loss: 0.0021419499535113573
Epoch 185 / 200 | iteration 160 / 171 | Total Loss: 3.6233317852020264 | KNN Loss: 3.606220006942749 | CLS Loss: 0.017111727967858315
Epoch 185 / 200 | iteration 170 / 171 | Total Loss: 3.6639420986175537 | KNN Loss: 3.6395184993743896 | CLS Loss: 0.024423567578196526
Epoch: 185, Loss: 3.6137, Train: 0.9960, Valid: 0.9851, Best: 0.9881
Epoch 186 / 200 | iteration 0 / 171 | Total Loss: 3.584944725036621 | KNN Loss: 3.572535514831543 | CLS Loss: 0.012409098446369171
Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 3.581165313720703 | KNN Loss: 3.576125383377075 | CLS Loss: 0.005039928946644068
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 3.6369593143463135 | KNN Loss: 3.624846935272217 | CLS Loss: 0.0121124750003218

Epoch 189 / 200 | iteration 20 / 171 | Total Loss: 3.6054458618164062 | KNN Loss: 3.597970724105835 | CLS Loss: 0.007475217338651419
Epoch 189 / 200 | iteration 30 / 171 | Total Loss: 3.614314079284668 | KNN Loss: 3.59910249710083 | CLS Loss: 0.015211476944386959
Epoch 189 / 200 | iteration 40 / 171 | Total Loss: 3.598947525024414 | KNN Loss: 3.5901756286621094 | CLS Loss: 0.008771778084337711
Epoch 189 / 200 | iteration 50 / 171 | Total Loss: 3.5944154262542725 | KNN Loss: 3.579150438308716 | CLS Loss: 0.015265066176652908
Epoch 189 / 200 | iteration 60 / 171 | Total Loss: 3.6110453605651855 | KNN Loss: 3.5961081981658936 | CLS Loss: 0.01493705902248621
Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 3.5712316036224365 | KNN Loss: 3.556103467941284 | CLS Loss: 0.01512810867279768
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 3.6456170082092285 | KNN Loss: 3.6373629570007324 | CLS Loss: 0.008254148066043854
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 3.6576714515686035 |

Epoch 192 / 200 | iteration 80 / 171 | Total Loss: 3.5929462909698486 | KNN Loss: 3.5877981185913086 | CLS Loss: 0.005148110445588827
Epoch 192 / 200 | iteration 90 / 171 | Total Loss: 3.596129894256592 | KNN Loss: 3.588235855102539 | CLS Loss: 0.007894113659858704
Epoch 192 / 200 | iteration 100 / 171 | Total Loss: 3.62921142578125 | KNN Loss: 3.6154186725616455 | CLS Loss: 0.013792778365314007
Epoch 192 / 200 | iteration 110 / 171 | Total Loss: 3.6082143783569336 | KNN Loss: 3.606281280517578 | CLS Loss: 0.0019331170478835702
Epoch 192 / 200 | iteration 120 / 171 | Total Loss: 3.6608004570007324 | KNN Loss: 3.642062187194824 | CLS Loss: 0.01873824931681156
Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 3.7065107822418213 | KNN Loss: 3.680169105529785 | CLS Loss: 0.02634158544242382
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 3.6399686336517334 | KNN Loss: 3.6156833171844482 | CLS Loss: 0.024285387247800827
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 3.60504770278

Epoch 195 / 200 | iteration 150 / 171 | Total Loss: 3.6351470947265625 | KNN Loss: 3.6327919960021973 | CLS Loss: 0.0023551343474537134
Epoch 195 / 200 | iteration 160 / 171 | Total Loss: 3.559770345687866 | KNN Loss: 3.5528485774993896 | CLS Loss: 0.0069217742420732975
Epoch 195 / 200 | iteration 170 / 171 | Total Loss: 3.5854344367980957 | KNN Loss: 3.576892375946045 | CLS Loss: 0.008541978895664215
Epoch: 195, Loss: 3.6105, Train: 0.9979, Valid: 0.9871, Best: 0.9881
Epoch 196 / 200 | iteration 0 / 171 | Total Loss: 3.591273784637451 | KNN Loss: 3.5829689502716064 | CLS Loss: 0.008304794318974018
Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 3.5628163814544678 | KNN Loss: 3.5591280460357666 | CLS Loss: 0.0036883633583784103
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 3.576659917831421 | KNN Loss: 3.5755321979522705 | CLS Loss: 0.0011278289603069425
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 3.5898005962371826 | KNN Loss: 3.5870003700256348 | CLS Loss: 0.0028003354

Epoch 199 / 200 | iteration 30 / 171 | Total Loss: 3.6455960273742676 | KNN Loss: 3.640336275100708 | CLS Loss: 0.005259797442704439
Epoch 199 / 200 | iteration 40 / 171 | Total Loss: 3.6235580444335938 | KNN Loss: 3.5912466049194336 | CLS Loss: 0.032311420887708664
Epoch 199 / 200 | iteration 50 / 171 | Total Loss: 3.664645195007324 | KNN Loss: 3.6504340171813965 | CLS Loss: 0.014211189933121204
Epoch 199 / 200 | iteration 60 / 171 | Total Loss: 3.5963103771209717 | KNN Loss: 3.581380605697632 | CLS Loss: 0.014929704368114471
Epoch 199 / 200 | iteration 70 / 171 | Total Loss: 3.5769057273864746 | KNN Loss: 3.571948766708374 | CLS Loss: 0.004956903867423534
Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 3.5752196311950684 | KNN Loss: 3.5630714893341064 | CLS Loss: 0.012148236855864525
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 3.600748062133789 | KNN Loss: 3.584953546524048 | CLS Loss: 0.015794595703482628
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 3.56090664863586

In [8]:
test(model, test_data_iter, device)

tensor(0.9870, device='cuda:0')

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [14]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9703074322781051


In [15]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [16]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [17]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [18]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [31]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [32]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [33]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 042 | Total loss: 1.627 | Reg loss: 0.014 | Tree loss: 1.627 | Accuracy: 0.072266 | 3.746 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 01 | Batch: 000 / 042 | Total loss: 1.587 | Reg loss: 0.006 | Tree loss: 1.587 | Accuracy: 0.429688 | 3.663 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
l

Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 18 | Batch: 000 / 042 | Total loss: 1.263 | Reg loss: 0.014 | Tree loss: 1.263 | Accuracy: 0.560547 | 3.886 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 19 | Batch: 000 / 042 | Total loss: 1.232 | Reg loss: 0.014 | Tree loss: 1.232 | Accuracy: 0.587891 | 3.888 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.984042

Epoch: 35 | Batch: 000 / 042 | Total loss: 1.214 | Reg loss: 0.015 | Tree loss: 1.214 | Accuracy: 0.607422 | 3.915 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 36 | Batch: 000 / 042 | Total loss: 1.184 | Reg loss: 0.015 | Tree loss: 1.184 | Accuracy: 0.615234 | 3.915 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 37 | Batch: 000 / 042 | Total loss: 1.249 | Reg l

Epoch: 53 | Batch: 000 / 042 | Total loss: 1.147 | Reg loss: 0.014 | Tree loss: 1.147 | Accuracy: 0.640625 | 3.924 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 54 | Batch: 000 / 042 | Total loss: 1.218 | Reg loss: 0.014 | Tree loss: 1.218 | Accuracy: 0.599609 | 3.924 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 55 | Batch: 000 / 042 | Total loss: 1.238 | Reg l

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")